### Set proper directory, Load packages and R Functions

In [ ]:
# You should have the R script 'sun_cai(2007)_est.R' in your working directory

In [9]:
import pandas as pd
import numpy as np 
from bed_reader import open_bed, sample_file
import time
import re
import os
import math 
import itertools as itertools
from itertools import product
import statsmodels.stats.multitest
from scipy.stats import norm
from scipy.sparse import lil_matrix
from scipy.io import savemat, loadmat
from scipy.linalg import inv
# Using R inside python
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.vectors import FloatVector
# Set working directory
os.chdir("E:\\GWAS\\plink_win64_20231211")
# Defining the R script and loading the instance in Python
r = robjects.r
r['source']('sun_cai(2007)_est.R')

# Loading the functions for sun and cai method defined in R.
est_sun_cai_r = robjects.globalenv['epsest.func']   #### function for estimating $\pi$

rej_sun_cai_r = robjects.globalenv['adaptZ.funcnull']

lcfdr_sun_cai_r = robjects.globalenv['adaptZ.funcnulllcfdr']
def locFDRS_GWAS_thread(Z,scov,B,b,pi,tau):
    K = np.size(Z);
    locFDR = np.zeros(K);
    for i in range(K):
        Z_sub = Z[max(i-B,0):(min(i+B,K-1)+1)];
        cov_sub = scov[max(i-B,0):(min(i+B,K-1)+1),max(i-B,0):(min(i+B,K-1)+1)].toarray();
        l = min(i+B,K-1) - max(i-B,0) + 1;
        H = product([0,1], repeat=l);                                           
        sum1 = sum2 = 0;
        for s in H:
            if(s[min(B,i)] == 0): 
                t = ((np.exp(-(1/2)*np.dot(np.inner((Z_sub - np.multiply(s,b)), 
                                         np.linalg.inv(cov_sub+np.multiply(np.diag(s),pow(tau,2)))),
                                  (Z_sub - np.multiply(s,b))))* 
                pow(pi,sum(s)) * pow(1-pi,l-sum(s)))/np.sqrt(np.linalg.det(cov_sub+np.multiply(np.diag(s),pow(tau,2)))));
                sum1 = sum1 + t;
                sum2 = sum2 + t;
            else:
                sum2 = sum2 + ((np.exp(-(1/2)*np.inner(np.dot(Z_sub - np.multiply(s,b), 
                                         np.linalg.inv(cov_sub+np.multiply(np.diag(s),pow(tau,2)))),
                                  (Z_sub - np.multiply(s,b))))* 
               pow(pi,sum(s)) * pow(1-pi,l-sum(s)))/np.sqrt(np.linalg.det(cov_sub+np.multiply(np.diag(s),pow(tau,2)))));
        locFDR[i]= sum1/sum2;
  
  
    return(locFDR);

def rejected(p_val,level):
    oo = np.argsort(p_val);
    ss = np.sort(p_val);
    stat = np.divide(np.cumsum(ss),np.arange(1,np.size(ss)+1,1));
    collection = np.where(stat <= level);
    return(oo[collection]);

### Generate data (y,X) by setting parameters of your own choice

In [ ]:
# For finding Z-scores and it's covariance, you need more sample size than the number of variables (n>p).
# If you have more variables than sample size (p>n), then use prior information (like short range dependence in GWAS) to find Z-scores and their covariance. One such strategy is given in our GWAS analysis as an example.
# This step is not needed if you already have real data with you in (y,X) format

In [287]:
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)


# Parameters
n = 3000  # Number of data points (rows of X)
p = 1000   # Number of predictors (columns of X)
rho = 0.8        # Autoregressive parameter
k = 60            # number of variables with nonzero coefficients
amplitude = 3.5   # signal amplitude (for noise level = 1)

# Step 1: Create the AR(1) covariance matrix
cov_matrix = rho ** np.abs(np.subtract.outer(np.arange(p), np.arange(p)))

# Step 2: Generate the design matrix X with rows from a multivariate normal distribution
X = np.random.multivariate_normal(mean=np.zeros(p), cov=cov_matrix, size=n)

# Step 3: Initialize the beta vector with nonzero values
beta = np.random.normal(0,0.1,p)

# Step 4: Randomly choose indices for zeros
zero_indices = np.random.choice(p, p-k, replace=False);

# Step 5: Set the chosen positions in beta to zero
beta[zero_indices] = 0;

# Step 6: Generate the error term epsilon (assume standard normal errors)
epsilon = np.random.randn(n)

# Step 7: Calculate the response variable y
y = X @ beta + epsilon

### Find Z-scores and their correlation

In [1]:
# Use your real data (y,X) here

In [291]:
X = (X - np.mean(X)) #/ np.std(X)
y = (y-np.mean(y))#/np.std(y)
XTX = np.matmul(np.transpose(np.array(X)),np.array(X));
XTXinv = np.linalg.inv(XTX);
XTy = np.matmul(np.transpose(np.array(X)),np.array(y));
betahat = np.matmul(XTXinv,XTy)
e =  y-np.matmul(np.array(X),betahat)
est_sigma = np.sqrt(np.dot(e,e)/(np.shape(X)[0]-np.shape(X)[1]))
# S_inv = np.diag(np.sqrt(1./np.diag(np.array(XTXinv))));
S_inv = np.diag((np.diag(XTXinv)**-0.5));
corrzscore = (np.matmul(np.matmul(S_inv,XTXinv),S_inv)); ### correlation matrix of z-scores
del XTXinv
del XTX
del e
del XTy
del X;
zscore = np.matmul(S_inv,betahat)/est_sigma ### required z-scores
del(S_inv);
del(betahat);

### estimate of non-null proportion by Jin and Cai (2008) Procedure

In [336]:
pi_est = max(est_sun_cai_r(FloatVector(zscore),0.,1.)+0.)
print(["estimate of non-null proportion",pi_est])
lcfdrsc = np.array(lcfdr_sun_cai_r(FloatVector(zscore)));

['estimate of non-null proportion', 0.04041045029197221]


### estimate other two parameters of two group model by EM algorithm

In [338]:
pi_est = max(est_sun_cai_r(FloatVector(zscore),0.,1.)+0.001);
maxiter =100;
B = 1;
est = np.zeros(2);
for i in range(B):
    est_mat = np.zeros(2*maxiter).reshape(maxiter ,2)
    z=zscore
    scovest = scovJ = lil_matrix((np.size(z), np.size(z)));
    scovest.setdiag(np.ones(np.size(z)),0);
    b_ini = np.mean(z)/pi_est; tausq_ini = np.amax([0.,(np.var(z)-1-(pow(b_ini,2)*(1-pow(pi_est,2))))/pi_est]); 
    est_mat[0,0] = b_ini; est_mat[0,1] = tausq_ini;
    for iter in 1+np.arange(maxiter-1):
      print(iter);
      p_tau = 1-locFDRS_GWAS_thread(z,scovest,0,b_ini,pi_est,np.sqrt(tausq_ini));
      #pi_next = np.mean(p_tau);
      b_next = (np.sum((p_tau)*z)/max(np.sum((p_tau)),0.01));
      tausq_next = np.amax([0,(np.sum((p_tau)*pow(z-b_next,2))
      /max(np.sum((p_tau)),0.01))-1]);
      b_ini = b_next; tausq_ini = tausq_next;
      est_mat[iter,0] = b_ini; est_mat[iter,1] = tausq_ini;
      if np.amax(abs(est_mat[iter,:]-est_mat[iter-1,:])) < pow(10,-5):
         print(est_mat[iter,:]);
         break;
    est = est+est_mat[iter,:]
print(["estimates of other parameters of two group model",est/B]);

1
2
3
4
5
6
7
8
9
10
11
[-0.59144951  5.74208638]
['estimates of other parameters of two group model', array([-0.59144951,  5.74208638])]


### Calculate locFDR statistics with estimated parameters

In [297]:
K = np.size(zscore);
N=5;
ScovJ = lil_matrix((K, K));
for n in range((2*N)+1):
    ScovJ.setdiag(corrzscore.diagonal(n),n);
    ScovJ.setdiag(corrzscore.diagonal(n),-n);

del corrzscore
lcfdr0 = locFDRS_GWAS_thread(zscore, ScovJ, 0, b_next, pi_est, np.sqrt(tausq_next));
lcfdr1 = locFDRS_GWAS_thread(zscore, ScovJ, 1, b_next, pi_est, np.sqrt(tausq_next));
lcfdr2 = locFDRS_GWAS_thread(zscore, ScovJ, 2, b_next, pi_est, np.sqrt(tausq_next));
lcfdr3 = locFDRS_GWAS_thread(zscore, ScovJ, 3, b_next, pi_est, np.sqrt(tausq_next));

In [316]:
alpha = 0.05; ### Set significance level

### Number of rejections

In [334]:
print(["BH procedure",np.size(np.where(statsmodels.stats.multitest.multipletests(2*(1-norm.cdf(abs(zscore))), 
                                                   alpha=(alpha/(1-0.)), method='fdr_bh', 
                                                         is_sorted=False, returnsorted=False)[0]==True)[0])])
print(["BH procedure",np.size(np.where(statsmodels.stats.multitest.multipletests(2*(1-norm.cdf(abs(zscore))), 
                                                   alpha=(alpha/(1-pi_est)), method='fdr_bh', 
                                                         is_sorted=False, returnsorted=False)[0]==True)[0])])
print(["Sun and Cai Procedure",np.size(rejected(lcfdrsc,alpha))])
print(["T0-rule",np.size(rejected(lcfdr0,alpha))])
print(["T1-rule",np.size(rejected(lcfdr1,alpha))])
print(["T2-rule",np.size(rejected(lcfdr2,alpha))])
print(["T3-rule",np.size(rejected(lcfdr3,alpha))])

['BH procedure', 7]
['BH procedure', 7]
['Sun and Cai Procedure', 7]
['T0-rule', 6]
['T1-rule', 14]
['T2-rule', 17]
['T3-rule', 17]


### Checking how many selected variables are truly non-null

In [ ]:
# You do not need this step for real data since you do not know the truth for real data

In [305]:
idxNonNull=np.where(beta!=0)[0]

In [344]:
print(["BH procedure",np.size(np.intersect1d(np.where(statsmodels.stats.multitest.multipletests(2*(1-norm.cdf(abs(zscore))), 
                                                   alpha=(alpha/(1-0.)), method='fdr_bh', 
                                                         is_sorted=False, returnsorted=False)[0]==True)[0],idxNonNull))])
print(["BH procedure",np.size(np.intersect1d(np.where(statsmodels.stats.multitest.multipletests(2*(1-norm.cdf(abs(zscore))), 
                                                   alpha=(alpha/(1-pi_est)), method='fdr_bh', 
                                                         is_sorted=False, returnsorted=False)[0]==True)[0],idxNonNull))])
print(["Sun and Cai Procedure",np.size(np.intersect1d(rejected(lcfdrsc,alpha),idxNonNull))])
print(["T0-rule",np.size(np.intersect1d(rejected(lcfdr0,alpha),idxNonNull))])
print(["T1-rule",np.size(np.intersect1d(rejected(lcfdr1,alpha),idxNonNull))])
print(["T2-rule",np.size(np.intersect1d(rejected(lcfdr2,alpha),idxNonNull))])
print(["T3-rule",np.size(np.intersect1d(rejected(lcfdr3,alpha),idxNonNull))])

['BH procedure', 7]
['BH procedure', 7]
['Sun and Cai Procedure', 7]
['T0-rule', 6]
['T1-rule', 13]
['T2-rule', 16]
['T3-rule', 17]
